# NHS Core

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [ ]:
import os
import csv

import common_core

## Runtime parameters

The "lazy" option skips files that already exist locally

In [ ]:
skipExisting = True

rawPath = os.path.join(common_core.projdir, "data", "nhs-statistics", "raw")

## Configuration

Data to download from the NHS statistical work area

In [ ]:
# Only using England for now due to the availability of age demographics, etc
nationNames = ["England"]

# All 7 NHS regions in England
nhsRegionNames = ["North West", "North East and Yorkshire", "Midlands",
                  "East of England", "London", "South East", "South West"]

# Combine all of these area types into a single list
areas = [("nation", ["England"]), ("nhsregion", nhsRegionNames)]

In [ ]:
deathUrl = "https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-daily-deaths/"
admissionUrl = "https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-hospital-activity/"
sitRepUrl = "https://www.england.nhs.uk/statistics/statistical-work-areas/uec-sitrep/urgent-and-emergency-care-daily-situation-reports-2020-21/"

In [ ]:
deathFiles = [
    ("weekly", "COVID-19-total-announced-deaths-.*-weekly-file\.xlsx$"), # place above daily file to be picked up first
    ("daily", "COVID-19-total-announced-deaths-.*\.xlsx$"),
    ("daily", "COVID-19-daily-announced-deaths-.*\.xlsx$")
]

admissionFiles = [
    ("daily", "COVID-19-daily-admissions-and-beds.*\.xlsx$"),
    ("weekly", "Weekly-covid-admissions-and-beds-publication.*\.xlsx$"),
    ("monthly", "Covid-Publication-.*\.xlsx$")
]

sitRepFiles = [
    ("weekly", "UEC-Daily-SitRep-Acute-Web-File-Timeseries.*\.xlsx$")

]

In [ ]:
ageDemographics = [
    '00_04', '05_09', '10_14', '15_19', '20_24', '25_29',
    '30_34', '35_39', '40_44', '45_49', '50_54', '55_59',
    '60_64', '65_69', '70_74', '75_79', '80_84', '85_89', '90+'
]

## Download Data

Download spreadsheets by parsing the HTML for suitable links

In [ ]:
def downloadDeaths():
    webDownload = common_core.WebDownload(skipExisting=skipExisting)
    files = webDownload.downloadFiles(rawPath, deathUrl, deathFiles, "deaths")

In [ ]:
def downloadAdmissions():
    webDownload = common_core.WebDownload(skipExisting=skipExisting)
    files = webDownload.downloadFiles(rawPath, admissionUrl, admissionFiles, "admissions")

In [ ]:
def downloadSitReps():
    webDownload = common_core.WebDownload(skipExisting=skipExisting)
    files = webDownload.downloadFiles(rawPath, sitRepUrl, sitRepFiles, "sitreps")